In [1]:
import numpy as np
from pathlib import Path
from allensdk.core.reference_space_cache import ReferenceSpaceCache

cache_dir = Path("../reference_space_cache")

# --- Parameters ---
VOXEL = 50   # µm (use 10, 25, 50...)

# --- Load CCF annotation and structure tree ---
# Manifest will be cached locally in this directory
rsc = ReferenceSpaceCache(resolution=VOXEL,
                          reference_space_key="annotation/ccf_2017",
                          manifest=str(cache_dir / "manifest.json"))

annot, meta = rsc.get_annotation_volume()
tree = rsc.get_structure_tree()

In [2]:
# Note: The axes order in the meta data 'space' was wrong.
print(annot.shape)
dict(meta)

(264, 160, 228)


{'type': 'unsigned int',
 'dimension': 3,
 'space': 'left-posterior-superior',
 'sizes': array([264, 160, 228]),
 'space directions': array([[50.,  0.,  0.],
        [ 0., 50.,  0.],
        [ 0.,  0., 50.]]),
 'kinds': ['domain', 'domain', 'domain'],
 'endian': 'little',
 'encoding': 'gzip',
 'space origin': array([0., 0., 0.])}

In [3]:
# Corrected CCF axis order should be: (AP, DV, ML) (per Allen docs)
# Directions: posterior, inferior, right
AP_AXIS, DV_AXIS, ML_AXIS = 0, 1, 2

In [4]:
# --- Get structure IDs for VISp layers ---
# Options: 'VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b'
layer_acronyms = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b']
structures = tree.get_structures_by_acronym(layer_acronyms)
id_map = {s['acronym']: s['id'] for s in structures}

In [5]:
ml_size = annot.shape[ML_AXIS]
ml_mid = ml_size // 2  # midline index

hemisphere = 'left'  # 'left' or 'right'

ml_slice = [slice(None)] * 3
ml_slice[ML_AXIS] = slice(0, ml_mid) if hemisphere == 'left' else slice(ml_mid, ml_size)
hemi_annot = annot[tuple(ml_slice)]

In [6]:
# --- Loop over layers, compute DV range ---
ranges = {}
for acr, sid in id_map.items():
    coords = np.argwhere(hemi_annot == sid)  # (AP, DV, ML)
    if coords.size == 0:
        continue

    ranges[acr]  = VOXEL * np.array([
        [coords[:,AP_AXIS].min(), coords[:,AP_AXIS].max()],
        [coords[:,DV_AXIS].min(), coords[:,DV_AXIS].max()],
        [coords[:,ML_AXIS].min(), coords[:,ML_AXIS].max()]
    ])


# Print results
print(f"{hemisphere.capitalize()} hemisphere (AP, DV, ML in rows; [min, max] in µm):\n")
for acr, mat in ranges.items():
    print(f"{acr}:\n{mat}\n")

Left hemisphere (AP, DV, ML in rows; [min, max] in µm):

VISp1:
[[ 7700 10350]
 [  400  1950]
 [ 2050  4350]]

VISp2/3:
[[ 7700 10250]
 [  500  1950]
 [ 2150  4300]]

VISp4:
[[ 7750 10100]
 [  700  2000]
 [ 2250  4250]]

VISp5:
[[ 7750 10050]
 [  750  2100]
 [ 2350  4250]]

VISp6a:
[[7750 9850]
 [1000 2250]
 [2500 4100]]

VISp6b:
[[7750 9600]
 [1150 2300]
 [2600 4100]]



In [7]:
def set_equal_3d_scaling(ax, x, y, z):
    """Set equal scaling for 3D plot"""
    # Get current ranges
    lim = np.array([[np.min(x), np.max(x)],
                    [np.min(y), np.max(y)],
                    [np.min(z), np.max(z)]])

    # Calculate centers
    center = np.mean(lim, axis=1, keepdims=True)

    # Find the cubic range
    half_range = np.max(lim[:, 1] - lim[:, 0]) / 2
    cubic_lim = center + half_range * np.array([-1, 1])

    # Set equal ranges around centers
    ax.set_xlim3d(cubic_lim[0])
    ax.set_ylim3d(cubic_lim[1])
    ax.set_zlim3d(cubic_lim[2])
    ax.set_box_aspect([1, 1, 1])
    return cubic_lim

In [8]:
import matplotlib.pyplot as plt

%matplotlib qt


colors = {
    'VISp1': 'cyan',
    'VISp2/3': 'blue',
    'VISp4': 'green',
    'VISp5': 'orange',
    'VISp6a': 'red',
    'VISp6b': 'purple'
}

scatter_density = 0.2

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

for acr, sid in id_map.items():
    mask = (hemi_annot == sid)
    coords = np.argwhere(mask)
    if coords.size == 0:
        continue

    # Convert voxel indices to microns
    AP = coords[:, AP_AXIS] * VOXEL
    DV = coords[:, DV_AXIS] * VOXEL
    ML = coords[:, ML_AXIS] * VOXEL
    # Plot only a subsample for speed
    n_voxels = coords.shape[0]
    idx = np.random.choice(n_voxels, size=int(n_voxels * scatter_density), replace=False)
    ax.plot(ML[idx], AP[idx], DV[idx], '.', color=colors[acr], markersize=2, label=acr, alpha=0.5)
    # ax.plot(ML, AP, DV, '.', color=colors[acr], markersize=2, label=acr, alpha=0.5)

ax.set_xlabel('ML (µm)')
ax.set_ylabel('AP (µm)')
ax.set_zlabel('DV (µm)')
ax.legend()
ax.set_title(f'{hemisphere.capitalize()} VISp Cortex')

data_lim = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
data_lim = set_equal_3d_scaling(ax, *data_lim)

ax.set_ylim3d(data_lim[1][::-1])  # flip AP
ax.set_zlim3d(data_lim[2][::-1])  # flip DV

plt.show()
